<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#WM" data-toc-modified-id="WM-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>WM</a></span></li><li><span><a href="#SC" data-toc-modified-id="SC-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>SC</a></span></li><li><span><a href="#Requests" data-toc-modified-id="Requests-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Requests</a></span></li></ul></div>

In [6]:
import pandas as pd
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)
pd.set_option('display.width', 1000)

import datetime
from datetime import date

# WM

In [18]:
all_wm_nt = pd.read_excel('../data/all_wm.xlsx', 'c_nt_abcs')
all_wm_metrics = pd.read_excel('../data/all_wm.xlsx', 'c_metrics')
all_wm_lift = pd.read_excel('../data/all_wm.xlsx', 'c_lift')
all_wm_ccontrol = pd.read_excel('../data/all_wm.xlsx', 'c_control_abc')

wes_non = pd.read_excel('../data/wm_tables_20200124.xlsx', 'nontraceable')
wes_trace = pd.read_excel('../data/wm_tables_20200124.xlsx', 'traceable')
wes_lift = pd.read_excel('../data/wm_tables_20200124.xlsx', 'lift')
wes_control = pd.read_excel('../data/wm_tables_20200124.xlsx', 'control')

wes_non_columns = wes_non.columns.to_list()
wes_trace_columns = wes_trace.columns.to_list()
wes_lift_columns = wes_lift.columns.to_list()
wes_control_columns = wes_control.columns.to_list()

all_wm_nt_mod = all_wm_nt.iloc[12:, :]
all_wm_nt_mod.columns = all_wm_nt_mod.iloc[0]
all_wm_nt_mod = all_wm_nt_mod.drop(all_wm_nt_mod.index[0])
all_wm_nt_mod = all_wm_nt_mod.rename(columns={'Rest of Supercenters stores': 'Rest of Chain stores'})

all_wm_metrics_mod = all_wm_metrics.iloc[12:, :]
all_wm_metrics_mod.columns = all_wm_metrics_mod.iloc[0]
all_wm_metrics_mod = all_wm_metrics_mod.drop(all_wm_metrics_mod.index[0])
all_wm_metrics_mod = all_wm_metrics_mod.rename(columns={'Rest of Supercenters Stores': 'Rest of Chain stores'})

all_wm_lift_mod = all_wm_lift.iloc[4:, :]
all_wm_lift_mod.columns = all_wm_lift_mod.iloc[0]
all_wm_lift_mod = all_wm_lift_mod.drop(all_wm_lift_mod.index[0])
all_wm_lift_mod = all_wm_lift_mod.rename(columns={'Rest of Supercenters': 'Control stores'})

all_wm_ccontrol_mod = all_wm_metrics.iloc[12:, :]
all_wm_ccontrol_mod.columns = all_wm_ccontrol_mod.iloc[0]
all_wm_ccontrol_mod = all_wm_ccontrol_mod.drop(all_wm_ccontrol_mod.index[0])
all_wm_ccontrol_mod = all_wm_ccontrol_mod.rename(columns={'Rest of Supercenters Stores': 'Rest of Chain stores'})

all_wm_nt_mod_ = all_wm_nt_mod[wes_non_columns]
all_wm_metrics_mod_ = all_wm_metrics_mod[wes_trace_columns]
all_wm_lift_mod_ = all_wm_lift_mod[wes_lift_columns]
all_wm_ccontrol_mod_ = all_wm_ccontrol_mod[wes_control_columns]

# writer = pd.ExcelWriter(f'output/wm_tables_{date.today()}.xlsx', engine='xlsxwriter')
# writer.save()
# writer = pd.ExcelWriter(f'output/wm_tables_{date.today()}.xlsx', engine='xlsxwriter')
# all_wm_nt_mod_.to_excel(writer, sheet_name='nontraceable')
# all_wm_metrics_mod_.to_excel(writer, sheet_name='traceable')
# all_wm_lift_mod_.to_excel(writer, sheet_name='lift')
# all_wm_ccontrol_mod_.to_excel(writer, sheet_name='control')
# writer.save()

In [279]:
from __future__ import print_function
import pickle
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request

import pygsheets

import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

import time
import os
import os.path
from os import path
from shutil import copy
import re

import logging
logging.basicConfig(filename='app.log', filemode='a', format='%(asctime)s - %(levelname)s - %(message)s', datefmt='%d-%b-%y %H:%M:%S')

# try: 
#     from config import mysql_credentials
#     from config import powerpoint_paths
# except ImportError:
#     logging.error('No config file found. Please create one', exc_info=True)

# https://pygsheets.readthedocs.io/en/stable/worksheet.html
def mysql_query(query='DEFAULT', verbose=False):
    import sqlalchemy as sql
    username = mysql_credentials['username']
    password = mysql_credentials['password']
    connection = mysql_credentials['connection']
    database_name = mysql_credentials['database_name']
    connect_string = f'mysql://{username}:{password}@{connection}/{database_name}'
    sql_engine = sql.create_engine(connect_string)
    df = pd.read_sql_query(query, sql_engine)
    if verbose: print(df.shape)
    return df

def worksheet(gsheet_url, network):
    gc = pygsheets.authorize(client_secret='credentials/client_secret.json')
    sh = gc.open_by_url(gsheet_url)

    network_dict = {
    "Sam's Club":0,
    'Walmart':1
    }
    
    network_choice = network_dict[network] # Relating network to sheet index
    # network_choice = 2 # testing
    wk = sh[network_choice]
    return wk

def metric_xlsx_migrater():
    sc_src = 'F:\Analytics\Longitudinals - Composite and Assembled'
    wm_src = 'F:\Analytics\Walmart Reporting\Output'
    dst = 'C:/Users/jay.yu/Documents/Git/case_list_refresh/data'

    files = []

    src_list = [wm_src, sc_src]

    for src in src_list:
        for r, d, f in os.walk(src):
            
            for file in f:
                if file == 'all_wm.xlsx':
                    print('Found all_wm, migrating now.')
                    path_string = os.path.join(r, file)
                    copy(path_string, dst)
                    
                if file == 'all_longitudinal_card.xlsx':
                    print('Found all_longitudinal_card.xlsx, migrating now.')
                    path_string = os.path.join(r, file)
                    copy(path_string, dst)
            break

# GOOGLE SPREADSHEET DATAFRAME UPDATER
def gsheet_import_df(wk, verbose=False):

    gsheet_df = wk.get_as_df()
    
    if verbose: print(df.shape)
    
    print(f'Successfully returned dataframe.')
    print(f'Imported Gsheet Dataframe has (rows, columns): {gsheet_df.shape}.')
    return gsheet_df

def gsheet_programs(gsheet_df):
    not_list = gsheet_df['Program'].to_list()
    return not_list

def all_wm_import():
    all_wm_metrics = pd.read_excel('../data/all_wm.xlsx', 'c_metrics')
    all_wm_metrics_mod = all_wm_metrics.iloc[12:, :]
    all_wm_metrics_mod.columns = all_wm_metrics_mod.iloc[0]
    all_wm_metrics_mod = all_wm_metrics_mod.drop(all_wm_metrics_mod.index[0])
    all_wm_metrics_mod = all_wm_metrics_mod.rename(columns={'Rest of Supercenters Stores': 'Rest of Chain stores'})

    all_wm_lift = pd.read_excel('../data/all_wm.xlsx', 'c_lift')
    all_wm_lift_mod = all_wm_lift.iloc[4:, :]
    all_wm_lift_mod.columns = all_wm_lift_mod.iloc[0]
    all_wm_lift_mod = all_wm_lift_mod.drop(all_wm_lift_mod.index[0])
    all_wm_lift_mod = all_wm_lift_mod.rename(columns={'Rest of Supercenters': 'Control stores'})

    wm_df_raw = all_wm_metrics_mod.merge(all_wm_lift_mod, how='left', left_on='Program', right_on='Program list', copy=False,suffixes=('', '_'))
    print('Successfully imported data from all_wm.xlsx.')
    return wm_df_raw

def all_longitudinal_card_import():
    sc_df_metrics = pd.read_excel('../data/all_longitudinal_card.xlsx', sheet_name='c_metrics')
    sc_df_metrics2 = sc_df_metrics.iloc[12:, :]
    sc_df_metrics2.columns = sc_df_metrics2.iloc[0]
    sc_df_metrics2 = sc_df_metrics2.drop(sc_df_metrics2.index[0])

    sc_df_lift = pd.read_excel('../data/all_longitudinal_card.xlsx', sheet_name='c_lift')
    sc_df_lift2 = sc_df_lift.iloc[4:, :]
    sc_df_lift2.columns = sc_df_lift2.iloc[0]
    sc_df_lift2 = sc_df_lift2.drop(sc_df_lift2.index[0])

    sc_df_raw = sc_df_metrics2.merge(sc_df_lift2, how='left', left_on='Program', right_on='Program list', copy=False, suffixes=('', '_'))
    
    sc_df_cfg = pd.read_excel('../data/all_longitudinal_card.xlsx', sheet_name='c_cfg') # Getting merchandising_type + placement_type
    sc_df_cfg = sc_df_cfg.iloc[5:, :]
    sc_df_cfg.columns = sc_df_cfg.iloc[0]
    sc_df_cfg = sc_df_cfg.drop(sc_df_cfg.index[0])
    sc_df_cfg = sc_df_cfg[['program_list', 'merchandising_type', 'placement_type']]

    sc_df_raw = sc_df_raw.merge(sc_df_cfg, how='left', left_on='Program', right_on='program_list', copy=False, suffixes=('', '_'))
    sc_df_raw = sc_df_raw.drop(columns=['start_date', 'end_date'])
    print('Successfully imported data from all_longitdinal_card.xlsx.')
    return sc_df_raw

def wm_formatter(wm_df_raw):
    # Formatting
    wm_df_raw['Traffic/store'] = wm_df_raw['Audience'] / wm_df_raw['Freeosk stores']
    wm_df_raw['Scans/store'] = wm_df_raw['Scans'] / wm_df_raw['Freeosk stores']
    wm_df_raw.loc[:, 'Tags'] = ''
    wm_df_raw.loc[:, 'Notes'] = ''
    wm_df_raw.columns = [column.replace('_', ' ') for column in list(wm_df_raw)]
    wm_df_raw.columns = [column.capitalize() for column in list(wm_df_raw)]
    wm_df_raw = wm_df_raw.rename(columns={'Non-traceable aud': 'Non traceable aud'})
    wm_df_raw['Program2'] = wm_df_raw['Program'].str.split("_", n=1, expand=True)[0]

    # Final column rearrangement
    rearranged_columns = ['Program', 'Tags', 'Notes', 'Weeks post', 'Start date', 'End date', 
                            'Item number', 'Placement name', 'Dept', 'Sub dept', 'Dept name', 
                            'Sub dept name', 'Freeosk stores', 'Rest of chain stores', 'Traceable audience', 
                            'Non traceable aud', 'Audience', 'Traffic/store', 'Scans', 'Scans/store', 'Total conv', 
                            'Repeat', 'A', 'B', 'C', 'Households', 'A%', 'B%', 'C%', 'A+b%', 'Control lift', 
                            'Freeosk lift', 'Program2']

    wm_df_raw = wm_df_raw[rearranged_columns].reset_index(drop=True)

    wm_df_raw = wm_df_raw.rename(columns={'A+b%':'A+B%'})

    wm_df_formatted = wm_df_raw[(wm_df_raw['Weeks post'] == '4W') & (wm_df_raw['Program'].str.contains('_4.zip'))]
    print(f'WM_Formatter completed! Shape is {wm_df_formatted.shape}')
    return wm_df_formatted

def sc_formatter(sc_df_raw):
    # Formatting
    sc_df_raw['Traffic/club'] = sc_df_raw['Audience'] / sc_df_raw['Freeosk clubs']
    sc_df_raw['Scans/club'] = sc_df_raw['Scans'] / sc_df_raw['Freeosk clubs']
    sc_df_raw['Program2'] = sc_df_raw['Program'].str.split("_", n=1, expand=True)[0]
    sc_df_raw['Unique Identifier'] = sc_df_raw['Program2'] + sc_df_raw['Item number'].astype(str)
    renamed_columns = {
        'HH_A%': 'A%',
        'HH_B%': 'B%',
        'HH_C%': 'C%'
    }
    sc_df_raw = sc_df_raw.rename(columns=renamed_columns)
    sc_df_raw.loc[:, 'Tags'] = ''
    sc_df_raw.loc[:, 'Notes'] = ''
    sc_df_raw.columns = [column.replace('_', ' ') for column in list(sc_df_raw)]
    sc_df_raw.columns = [column.capitalize() for column in list(sc_df_raw)]
    print(sc_df_raw.columns)
    # Mapping to food boolean
    food_mapping = pd.read_excel('../data/Food_Mapping.xlsx')
    sc_df_raw_2 = pd.merge(sc_df_raw, food_mapping,  how='left', 
                       left_on=['Category num','Subcat num'], 
                       right_on = ['CATEGORY_NUMBER','SUB_CATEGORY_NUMBER'])
    print(f'Successfully joined to Food_Mapping.xslx')
    print(sc_df_raw_2.columns)
    # Final column rearrangement
    rearranged_columns = ['Program', 'Tags', 'Notes', 'Weeks post', 'Start date', 
                      'End date', 'Item number', 'Item description', 'Category', 
                      'Subcategory', 'Category num', 'Subcat num', 'Freeosk clubs', 
                      'Control clubs', 'Transactions', 'Audience', 'Traffic/club', 
                      'Scans', 'Scans/club', 'Engagement', 'Conversion', 'Repeat', 
                      'Immediate', 'A%', 'B%', 'C%', 'Freeosk lift', 'Control lift', 
                      'Merchandising type', 'Placement type', 'Program2', 'Unique identifier', 'Food']

    sc_df_raw_3 = sc_df_raw_2[rearranged_columns].reset_index(drop=True)
    print(sc_df_raw_3.columns)
    sc_df_formatted = sc_df_raw_3[(sc_df_raw_3['Weeks post'] == '12W') & (sc_df_raw_3['Program'].str.contains('_12.zip'))]
    print(f'SC_Formatter completed! Shape is {sc_df_formatted.shape}')
    return sc_df_formatted

def gsheet_uploader(wk, gsheet_df, append_df):
    gsheet_import_appended = gsheet_df.append(append_df, ignore_index=True)
    
    wk.clear('A2') # A2 is start of dataframe
    wk.set_dataframe(gsheet_import_appended, 'A2', copy_index=False, copy_head=False, extend=False, fit=True, escape_formulae=True, nan='')
    print('New data has been successfully uploaded!') 

# HYPERLINK FUNCTIONS
def gfile_list_agg():

    # If modifying these scopes, delete the file token.pickle.
    SCOPES = ['https://www.googleapis.com/auth/drive.metadata.readonly']

    """Shows basic usage of the Drive v3 API.
    Prints the names and ids of the first 10 files the user has access to.
    """
    creds = None
    # The file token.pickle stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if os.path.exists('credentials/token.pickle'):
        with open('credentials/token.pickle', 'rb') as token:
            creds = pickle.load(token)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials/credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open('credentials/token.pickle', 'wb') as token:
            pickle.dump(creds, token)

    service = build('drive', 'v3', credentials=creds)

    # Call the Drive v3 API

    page_token = None
    file_list = {}
    page = 0

    while True:
        results = service.files().list(q="'1qQZpAhzmKoR7drBYh8kYWVS6GaMPCxr3' in parents", spaces='drive',
                                              fields='nextPageToken, files(id, name)',
                                              pageToken=page_token).execute()
        items = results.get('files', [])

        for item in items:
            file_list[item['name']] = item['id']

        page_token = results.get('nextPageToken', None)
        if page_token is None: break

    print('Google Drive: PPTX files collected')
    print(f'There are {len(file_list)} PPTX files in the Cases folder.')
    return file_list

def hyperlink_updater(wk, file_list):
    
    n_rows = wk.rows + 1
    for i in range(2, n_rows):
        
        cell = pygsheets.Cell(f'A{i}', worksheet=wk, cell_data=None)
        cell_value = wk.get_value(f'A{i}')
        
        if not cell.formula and cell_value: # Must have value but no hyperlink

            # print(f'Cell A{i} requires a hyperlink: {cell_value}')
            
            file_name = cell_value.split('.')[0] + '.pptx' # Name to help locate file names
            cell_nbr = f'A{i}'
                        
            file_list_names = list(file_list.keys())
            
            if file_name in file_list_names:
                print(f'Found one {file_name}!')

                file_name_zip = file_name.split('.pptx')[0] + '.zip' # Reference back to original name

                cell.formula = f'=HYPERLINK("https://drive.google.com/file/d/{file_list[file_name]}", "{file_name_zip}")'

                print(f'Replaced {cell_nbr}: {file_name} : {file_list[file_name]}')
                
            else: 
                print(f'COULD NOT REPLACE {cell_nbr}: {file_name}')

In [290]:
test1 = pd.read_excel('../data/test.xlsx', sheet_name = 'Sheet1')
test2 = pd.read_excel('../data/test.xlsx', sheet_name = 'Sheet2')


In [291]:
test1

,program
0,FSC18018_190736_20190524_12.zip
1,FSC18018_190736_20190628_12.zip
2,FSC18018_705387_20190524_12.zip
3,FSC18018_705387_20190628_12.zip
4,FSC18018_709743_20190524_12.zip
5,FSC18018_709743_20190628_12.zip
6,FSC18024_980099919_20180413_12.zip
7,FSC18054_434023_20181130_12.zip
8,FSC18054_902467_20181130_12.zip
9,FSC18055_980074723_20181102_12.zip


In [292]:
test2

,program,tags,notes
0,FSC19006_247824_20190308_12.zip,High repeat; Low engagement; High C%; Low lift,Low lift and engagement as it's a niche sub ca...
1,FSC18026_729234_20180427_12.zip,NaN,NaN
2,FSC18124_980135711_20190322_12.zip,NaN,NaN
3,FSC18025_729234_20180427_12.zip,High engagement; High C%; High repeat,Relatively low lift but high C% with high repe...
4,FSC18075_57091_20181005_12.zip,Comparable with ISB; High repeat; low engageme...,NaN
5,FSC19022_793064_20190510_12.zip,"High conversion; high repeat, low engagement, ...",Multiple Freeosk week peeks visible in sales t...
6,FSC19006_151618_20190308_12.zip,NaN,NaN
7,FSC18090_980120051_20181221_12.zip,NaN,NaN
8,FSC18081_728178_20181012_12.zip,NaN,NaN
9,FSC18018_705387_20180525_12.zip,NaN,NaN


In [294]:
test3 = test1.merge(test2, how='left')
test3.to_excel('CHECK.xlsx')

In [288]:
sc_df_raw['Start date']= pd.to_datetime(sc_df_raw['Start date'])
sc_df_raw['End date']= pd.to_datetime(sc_df_raw['End date'])
sc_df_raw

,Number,Program,Weeks post,Start date,End date,Item number,Item description,Tracker description,Short description,Sample name,Short sample name,Category,Subcategory,Category Num,Subcat Num,Freeosk clubs,Control clubs,Audience,Transactions,Scans,Unique Scans,Engagement,Immediate Conv,Wks 1-4 Conv,Wks 5-8 Conv,Wks 9-12 Conv,Delayed Conv,Total Conv,Repeat Conv,Conversion,Repeat,Immediate,Delayed,PW A,1-4 A,5-8 A,9-12 A,PW B,1-4 B,5-8 B,9-12 B,PW C,1-4 C,5-8 C,9-12 C,A,B,C,Households,A%,B%,C%,A+B%,PW A,1-4 A,5-8 A,9-12 A,PW B,1-4 B,5-8 B,9-12 B,PW C,1-4 C,5-8 C,9-12 C,A $,B $,C $,Dollars,A $ %,B $ %,C $ %,A delay $,B delay $,C delay $,PW A,1-4 A,5-8 A,9-12 A,PW B,1-4 B,5-8 B,9-12 B,PW C,1-4 C,5-8 C,9-12 C,A units,B units,C units,Units,Program list,First row,Last row,Num rows,start_date_,Start date_,Week-1,Week-2,Week-3,Week-4,Freeosk Program Week,Week-1,Week-2,Week-3,Week-4,Non-Freeosk Program Week,Week-1,Week-2,Week-3,Week-4,Freeosk Prior 4,Control Prior 4,Freeosk Prior 4 per week,Control Prior 4 per week,Freeosk lift,Control lift,Freeosk clubs_,Control clubs_,F PW per club,F Prev4 per club,C PW per club,C Prev4 per club,program_list,merchandising_type,placement_type
0,0,FSC18018_190736_20190524_0.zip,PW,2019-05-24,2019-05-30,190736,AIRHEADS BITES,Airheads Fruit Bites,Airheads Fruit Bites,Airheads Variety Pack,Airheads Variety Pack,CANDY,CT SUGAR,1,3,569,15,10795394,6673144,797427,783380,0.119498,542,0,0,0,0,542,10,0.000691874,0.0184502,1,0,273,0,0,0,117,0,0,0,152,0,0,0,273,117,152,542,0.50369,0.215867,0.280443,0.719557,3987.99,0,0,0,1890.34,0,0,0,3147.36,0,0,0,3987.99,1890.34,3147.36,9025.69,0.441849,0.20944,0.348711,0,0,0,282,0,0,0,134,0,0,0,222,0,0,0,282,134,222,638,FSC18018_190736_20190524_0.zip,1,741,740,'2019-05-24',2019-05-24 00:00:00,2019-05-17 00:00:00,2019-05-10 00:00:00,2019-05-03 00:00:00,2019-04-26 00:00:00,48845,52521.8,54215,56431.9,57935.4,1611.34,2070.28,1829.22,1999.38,1971.02,221104,7869.9,55276,1967.47,-0.116344,-0.181011,569,15,85.8435,97.1459,107.423,131.165,FSC18018_190736_20190524_0.zip,Tier 1 - Merchandised,Sample
1,0,FSC18018_190736_20190524_0.zip,4W,2019-05-24,2019-05-30,190736,AIRHEADS BITES,Airheads Fruit Bites,Airheads Fruit Bites,Airheads Variety Pack,Airheads Variety Pack,CANDY,CT SUGAR,1,3,569,15,10795394,6673144,797427,783380,0.119498,542,0,0,0,0,542,10,0.000691874,0.0184502,1,0,273,0,0,0,117,0,0,0,152,0,0,0,273,117,152,542,0.50369,0.215867,0.280443,0.719557,3987.99,0,0,0,1890.34,0,0,0,3147.36,0,0,0,3987.99,1890.34,3147.36,9025.69,0.441849,0.20944,0.348711,0,0,0,282,0,0,0,134,0,0,0,222,0,0,0,282,134,222,638,FSC18018_190736_20190524_0.zip,1,741,740,'2019-05-24',2019-05-24 00:00:00,2019-05-17 00:00:00,2019-05-10 00:00:00,2019-05-03 00:00:00,2019-04-26 00:00:00,48845,52521.8,54215,56431.9,57935.4,1611.34,2070.28,1829.22,1999.38,1971.02,221104,7869.9,55276,1967.47,-0.116344,-0.181011,569,15,85.8435,97.1459,107.423,131.165,FSC18018_190736_20190524_0.zip,Tier 1 - Merchandised,Sample
2,0,FSC18018_190736_20190524_0.zip,12W,2019-05-24,2019-05-30,190736,AIRHEADS BITES,Airheads Fruit Bites,Airheads Fruit Bites,Airheads Variety Pack,Airheads Variety Pack,CANDY,CT SUGAR,1,3,569,15,10795394,6673144,797427,783380,0.119498,542,0,0,0,0,542,10,0.000691874,0.0184502,1,0,273,0,0,0,117,0,0,0,152,0,0,0,273,117,152,542,0.50369,0.215867,0.280443,0.719557,3987.99,0,0,0,1890.34,0,0,0,3147.36,0,0,0,3987.99,1890.34,3147.36,9025.69,0.441849,0.20944,0.348711,0,0,0,282,0,0,0,134,0,0,0,222,0,0,0,282,134,222,638,FSC18018_190736_20190524_0.zip,1,741,740,'2019-05-24',2019-05-24 00:00:00,2019-05-17 00:00:00,2019-05-10 00:00:00,2019-05-03 00:00:00,2019-04-26 00:00:00,48845,52521.8,54215,56431.9,57935.4,1611.34,2070.28,1829.22,1999.38,1971.02,221104,7869.9,55276,1967.47,-0.116344,-0.181011,569,15,85.8435,97.1459,107.423,131.165,FSC18018_190736_20190524_0.zip,Tier 1 - Merchandised,Sample
3,1,FSC18018_190736_20190524_12.zip,PW,2019-05-24,2019-05-30,190736,AIRHEADS BITES,Airheads Fruit Bites,Airheads Fruit Bites,Airheads Variety Pack,Ai

In [283]:
n_rows = wk.rows + 1
for i in range(2, n_rows):

    cell = pygsheets.Cell(f'A{i}', worksheet=wk, cell_data=None)
    cell_value = wk.get_value(f'A{i}')

    if not cell.formula and cell_value: # Must have value but no hyperlink

        # print(f'Cell A{i} requires a hyperlink: {cell_value}')

        file_name = cell_value.split('.')[0] + '.pptx' # Name to help locate file names
        cell_nbr = f'A{i}'

#         file_list_names = list(file_list.keys())
        print(file_name)

FSC18018_190736_20190524_12.pptx
FSC18018_190736_20190628_12.pptx
FSC18018_705387_20190524_12.pptx
FSC18018_705387_20190628_12.pptx
FSC18018_709743_20190524_12.pptx
FSC18018_709743_20190628_12.pptx
FSC18024_980099919_20180413_12.pptx
FSC18054_434023_20181130_12.pptx


KeyboardInterrupt: 

In [259]:
gsheet_url = 'https://docs.google.com/spreadsheets/d/1wsnBd3AHObl4gnUJ2MlwkusuXRoOsQpu2Kx6zGH8bVY/edit#gid=0' # TEST

network_name_sub = "Sam's Club"

print("Running appender for Sam's Club.")
wk = worksheet(gsheet_url, network_name_sub)
gsheet_df = gsheet_import_df(wk)


Running appender for Sam's Club.
Successfully returned dataframe.


In [265]:
for x in append_df.columns:
    if x not in gsheet_df.columns:
        print(x)

In [263]:
gsheet_df.columns

Index(['Program', 'Tags', 'Notes', 'Weeks post', 'Start date', 'End date', 'Item number', 'Item description', 'Category', 'Subcategory', 'Category num', 'Subcat num', 'Freeosk clubs', 'Control clubs', 'Transactions', 'Audience', 'Traffic/club', 'Scans', 'Scans/club', 'Engagement', 'Conversion', 'Repeat', 'Immediate', 'A%', 'B%', 'C%', 'Freeosk lift', 'Control lift', 'Merchandising type', 'Placement type', 'Program2', 'Unique identifier', 'Food'], dtype='object')

In [261]:
gsheet_df.shape

(1182, 33)

In [281]:
sc_df_raw

,Number,Program,Weeks post,Start date,End date,Item number,Item description,Tracker description,Short description,Sample name,Short sample name,Category,Subcategory,Category Num,Subcat Num,Freeosk clubs,Control clubs,Audience,Transactions,Scans,Unique Scans,Engagement,Immediate Conv,Wks 1-4 Conv,Wks 5-8 Conv,Wks 9-12 Conv,Delayed Conv,Total Conv,Repeat Conv,Conversion,Repeat,Immediate,Delayed,PW A,1-4 A,5-8 A,9-12 A,PW B,1-4 B,5-8 B,9-12 B,PW C,1-4 C,5-8 C,9-12 C,A,B,C,Households,A%,B%,C%,A+B%,PW A,1-4 A,5-8 A,9-12 A,PW B,1-4 B,5-8 B,9-12 B,PW C,1-4 C,5-8 C,9-12 C,A $,B $,C $,Dollars,A $ %,B $ %,C $ %,A delay $,B delay $,C delay $,PW A,1-4 A,5-8 A,9-12 A,PW B,1-4 B,5-8 B,9-12 B,PW C,1-4 C,5-8 C,9-12 C,A units,B units,C units,Units,Program list,First row,Last row,Num rows,start_date_,Start date_,Week-1,Week-2,Week-3,Week-4,Freeosk Program Week,Week-1,Week-2,Week-3,Week-4,Non-Freeosk Program Week,Week-1,Week-2,Week-3,Week-4,Freeosk Prior 4,Control Prior 4,Freeosk Prior 4 per week,Control Prior 4 per week,Freeosk lift,Control lift,Freeosk clubs_,Control clubs_,F PW per club,F Prev4 per club,C PW per club,C Prev4 per club,program_list,merchandising_type,placement_type
0,0,FSC18018_190736_20190524_0.zip,PW,2019-05-24 00:00:00,2019-05-30 00:00:00,190736,AIRHEADS BITES,Airheads Fruit Bites,Airheads Fruit Bites,Airheads Variety Pack,Airheads Variety Pack,CANDY,CT SUGAR,1,3,569,15,10795394,6673144,797427,783380,0.119498,542,0,0,0,0,542,10,0.000691874,0.0184502,1,0,273,0,0,0,117,0,0,0,152,0,0,0,273,117,152,542,0.50369,0.215867,0.280443,0.719557,3987.99,0,0,0,1890.34,0,0,0,3147.36,0,0,0,3987.99,1890.34,3147.36,9025.69,0.441849,0.20944,0.348711,0,0,0,282,0,0,0,134,0,0,0,222,0,0,0,282,134,222,638,FSC18018_190736_20190524_0.zip,1,741,740,'2019-05-24',2019-05-24 00:00:00,2019-05-17 00:00:00,2019-05-10 00:00:00,2019-05-03 00:00:00,2019-04-26 00:00:00,48845,52521.8,54215,56431.9,57935.4,1611.34,2070.28,1829.22,1999.38,1971.02,221104,7869.9,55276,1967.47,-0.116344,-0.181011,569,15,85.8435,97.1459,107.423,131.165,FSC18018_190736_20190524_0.zip,Tier 1 - Merchandised,Sample
1,0,FSC18018_190736_20190524_0.zip,4W,2019-05-24 00:00:00,2019-05-30 00:00:00,190736,AIRHEADS BITES,Airheads Fruit Bites,Airheads Fruit Bites,Airheads Variety Pack,Airheads Variety Pack,CANDY,CT SUGAR,1,3,569,15,10795394,6673144,797427,783380,0.119498,542,0,0,0,0,542,10,0.000691874,0.0184502,1,0,273,0,0,0,117,0,0,0,152,0,0,0,273,117,152,542,0.50369,0.215867,0.280443,0.719557,3987.99,0,0,0,1890.34,0,0,0,3147.36,0,0,0,3987.99,1890.34,3147.36,9025.69,0.441849,0.20944,0.348711,0,0,0,282,0,0,0,134,0,0,0,222,0,0,0,282,134,222,638,FSC18018_190736_20190524_0.zip,1,741,740,'2019-05-24',2019-05-24 00:00:00,2019-05-17 00:00:00,2019-05-10 00:00:00,2019-05-03 00:00:00,2019-04-26 00:00:00,48845,52521.8,54215,56431.9,57935.4,1611.34,2070.28,1829.22,1999.38,1971.02,221104,7869.9,55276,1967.47,-0.116344,-0.181011,569,15,85.8435,97.1459,107.423,131.165,FSC18018_190736_20190524_0.zip,Tier 1 - Merchandised,Sample
2,0,FSC18018_190736_20190524_0.zip,12W,2019-05-24 00:00:00,2019-05-30 00:00:00,190736,AIRHEADS BITES,Airheads Fruit Bites,Airheads Fruit Bites,Airheads Variety Pack,Airheads Variety Pack,CANDY,CT SUGAR,1,3,569,15,10795394,6673144,797427,783380,0.119498,542,0,0,0,0,542,10,0.000691874,0.0184502,1,0,273,0,0,0,117,0,0,0,152,0,0,0,273,117,152,542,0.50369,0.215867,0.280443,0.719557,3987.99,0,0,0,1890.34,0,0,0,3147.36,0,0,0,3987.99,1890.34,3147.36,9025.69,0.441849,0.20944,0.348711,0,0,0,282,0,0,0,134,0,0,0,222,0,0,0,282,134,222,638,FSC18018_190736_20190524_0.zip,1,741,740,'2019-05-24',2019-05-24 00:00:00,2019-05-17 00:00:00,2019-05-10 00:00:00,2019-05-03 00:00:00,2019-04-26 00:00:00,48845,52521.8,54215,56431.9,57935.4,1611.34,2070.28,1829.22,1999.38,1971.02,221104,7869.9,55276,1967.47,-0.116344,-0.181011,569,15,85.8435,97.1459,107.423,131.165,FSC18018_190736_20190524_0.zip,Tier 1 - Merchandised,Sample
3,1,FSC18018_190736_20190524_12.zip,PW,2019-05-24 00:00:00,2019-05-30 00:00:00,190736,AIRHEADS 

In [280]:
sc_df_raw = all_longitudinal_card_import()


Successfully imported data from all_longitdinal_card.xlsx.


In [276]:
sc_df_formatted = sc_formatter(sc_df_raw)
not_list = gsheet_df['Program'].to_list() # Checks for rows to append
append_df = sc_df_formatted[~sc_df_formatted['Program'].isin(not_list)] 

Index(['Number', 'Program', 'Weeks post', 'Start date', 'End date', 'Start date', 'End date', 'Item number', 'Item description', 'Tracker description',
       ...
       'C prev4 per club', 'Program list', 'Merchandising type', 'Placement type', 'Traffic/club', 'Scans/club', 'Program2', 'Unique identifier', 'Tags', 'Notes'], dtype='object', length=134)
Successfully joined to Food_Mapping.xslx
Index(['Number', 'Program', 'Weeks post', 'Start date', 'End date', 'Start date', 'End date', 'Item number', 'Item description', 'Tracker description',
       ...
       'Scans/club', 'Program2', 'Unique identifier', 'Tags', 'Notes', 'cat_description', 'Food', 'subcat_description', 'CATEGORY_NUMBER', 'SUB_CATEGORY_NUMBER'], dtype='object', length=139)
Index(['Program', 'Tags', 'Notes', 'Weeks post', 'Start date', 'Start date', 'End date', 'End date', 'Item number', 'Item description', 'Category', 'Subcategory', 'Category num', 'Subcat num', 'Freeosk clubs', 'Control clubs', 'Transactions', 'Audien

In [267]:
append_df

,Program,Tags,Notes,Weeks post,Start date,Start date,End date,End date,Item number,Item description,Category,Subcategory,Category num,Subcat num,Freeosk clubs,Control clubs,Transactions,Audience,Traffic/club,Scans,Scans/club,Engagement,Conversion,Repeat,Immediate,A%,B%,C%,Freeosk lift,Control lift,Merchandising type,Placement type,Program2,Unique identifier,Food
56,FSC18024_980099919_20180413_12.zip,,,12W,'2018-04-13',2018-04-13 00:00:00,NaN,2018-04-19 00:00:00,980099919,DOVE BODY POLISH,HEALTH AND BEAUTY AIDS,Bodywash,2,20,334,246,3817663,6018281,18018.8,371119,1111.13,0.097211,0.00528757,0.0747038,0.424008,0.698609,0.296239,0.00515198,6.13985,6.46672,Tier 1 - Other,Sample + TIS,FSC18024,FSC18024980099919,No
137,FSC18062_326282_20190809_12.zip,,,12W,'2019-08-09',2019-08-09 00:00:00,'2019-08-15',2019-08-15 00:00:00,326282,BLUE STOP PAIN CREAM,OTC,Analgesics,54,1,150,350,1726289,2744126,18294.2,136926,912.84,0.0793181,0.0166119,0.0675918,0.341987,0.693823,0.258729,0.0474485,2.38101,0.0198554,Tier 1 - Sampled,Sample,FSC18062,FSC18062326282,OTC
281,FSC18087_980136022_20191004_12.zip,,,12W,'2019-10-04',2019-10-04 00:00:00,'2019-10-10',2019-10-10 00:00:00,980136022,JERG ULTRA HEALING,HEALTH AND BEAUTY AIDS,Skin Care,2,13,200,387,2388378,3780148,18900.7,56087,280.435,0.0234833,0.140153,0.100831,0.820319,0.704792,0.0572524,0.237955,1.23152,0.0340662,Tier 1 - Merchandised,Non Dispense TIS,FSC18087,FSC18087980136022,No
317,FSC18090_980120051_20190906_12.zip,,,12W,'2019-09-06',2019-09-06 00:00:00,'2019-09-12',2019-09-12 00:00:00,980120051,DOWNY UNSTOP FRESH,LAUNDRY AND HOME CARE,Scent Boosters,13,1,569,20,6648855,10712566,18827,641829,1127.99,0.0965323,0.077352,0.229164,0.506541,0.445266,0.133977,0.420757,0.34317,0.142887,Tier 1 - Sampled,Sample,FSC18090,FSC18090980120051,No
569,FSC18118_326282_20191004_12.zip,,,12W,'2019-10-04',2019-10-04 00:00:00,'2019-10-10',2019-10-10 00:00:00,326282,BLUE STOP PAIN CREAM,OTC,Analgesics,54,1,148,353,1707021,2670404,18043.3,118213,798.736,0.0692511,0.0208258,0.0622424,0.525144,0.695383,0.230833,0.073784,2.75201,-0.0555159,Tier 1 - Sampled,Sample + OSO TIS (Sample Match),FSC18118,FSC18118326282,OTC
941,FSC19008_584542_20190816_12.zip,,,12W,'2019-08-16',2019-08-16 00:00:00,'2019-08-22',2019-08-22 00:00:00,584542,VITAFUSION WOMEN'S,OTC,Adult MultiVitamins,54,8,589,0,6988349,11253619,19106.3,535424,909.039,0.0766167,0.0216708,0.0414073,0.567073,0.705638,0.112585,0.181777,1.07828,NaN,Tier 1 - Sampled,Sample,FSC19008,FSC19008584542,OTC
950,FSC19008_624083_20190816_12.zip,,,12W,'2019-08-16',2019-08-16 00:00:00,'2019-08-22',2019-08-22 00:00:00,624083,VITAFUSION MEN'S,OTC,Adult MultiVitamins,54,8,589,0,6988349,11253619,19106.3,535424,909.039,0.0766167,0.00704375,0.0405218,0.434638,0.638079,0.147377,0.214543,0.686076,NaN,Tier 1 - Merchandised,Sample,FSC19008,FSC19008624083,OTC
959,FSC19008_980085899_20190816_12.zip,,,12W,'2019-08-16',2019-08-16 00:00:00,'2019-08-22',2019-08-22 00:00:00,980085899,LIL CRITTERS GUMMY,OTC,Child Multivitamins,54,10,589,0,6988349,11253619,19106.3,535424,909.039,0.0766167,0.0164452,0.0398241,0.600214,0.740728,0.11757,0.141702,1.92476,NaN,Tier 1 - Merchandised,Sample,FSC19008,FSC19008980085899,OTC
983,FSC19010_980186688_20191115_12.zip,,,12W,'2019-11-15',2019-11-15 00:00:00,'2019-11-21',2019-11-21 00:00:00,980186688,TIDE PODS,LAUNDRY AND HOME CARE,Unit Dose Laundry,13,11,114,475,1292024,2016278,17686.6,119645,1049.52,0.0926028,0.03276,0.121275,0.323918,0.447266,0.185022,0.367712,0.129984,0.0166765,Tier 1 - Sampled,Sample,FSC19010,FSC19010980186688,No
1070,FSC19015_793064_20191025_12.zip,,,12W,'2019-10-25',2019-10-25 00:00:00,'2019-10-31',2019-10-31 00:00:00,793064,ENSUENO SPRING FRESH,LAUNDRY AND HOME CARE,Fabric Liquid Enhancers,13,70,100,26,1314492,2146944,21469.4,109223,1092.23,0.0830914,0.0742666,0.253512,0.427667,0.353246,0.184027,0.462726,0.13608,-0.155736,Tier 1 - Sampled,Sample,FSC19015,FSC19015793064,No


In [266]:
append_df.columns

Index(['Program', 'Tags', 'Notes', 'Weeks post', 'Start date', 'Start date', 'End date', 'End date', 'Item number', 'Item description', 'Category', 'Subcategory', 'Category num', 'Subcat num', 'Freeosk clubs', 'Control clubs', 'Transactions', 'Audience', 'Traffic/club', 'Scans', 'Scans/club', 'Engagement', 'Conversion', 'Repeat', 'Immediate', 'A%', 'B%', 'C%', 'Freeosk lift', 'Control lift', 'Merchandising type', 'Placement type', 'Program2', 'Unique identifier', 'Food'], dtype='object')

In [254]:
gsheet_df.shape

(1182, 33)

In [249]:
# all_wm_metrics = pd.read_excel('../data/all_wm.xlsx', 'c_metrics')
# all_wm_metrics_mod = all_wm_metrics.iloc[12:, :]
# all_wm_metrics_mod.columns = all_wm_metrics_mod.iloc[0]
# all_wm_metrics_mod = all_wm_metrics_mod.drop(all_wm_metrics_mod.index[0])
# all_wm_metrics_mod = all_wm_metrics_mod.rename(columns={'Rest of Supercenters Stores': 'Rest of Chain stores'})

# all_wm_lift = pd.read_excel('../data/all_wm.xlsx', 'c_lift')
# all_wm_lift_mod = all_wm_lift.iloc[4:, :]
# all_wm_lift_mod.columns = all_wm_lift_mod.iloc[0]
# all_wm_lift_mod = all_wm_lift_mod.drop(all_wm_lift_mod.index[0])
# all_wm_lift_mod = all_wm_lift_mod.rename(columns={'Rest of Supercenters': 'Control stores'})
# all_wm_lift_mod
wm_df_raw = all_wm_metrics_mod.merge(all_wm_lift_mod, how='left', left_on='Program', right_on='Program list', copy=False,suffixes=('', '_'))
# print('Successfully imported data from all_wm.xlsx.')
# return wm_df_raw


In [245]:
all_wm_import()

KeyError: 'Program list'

In [ ]:
# EXECUTION
# metric_xlsx_importer()
# wm_df_raw = all_wm_import()
# wm_df_formatted = wm_formatter(wm_df_raw)
# not_list = gsheet_programs(gsheet_df)
# append_df = wm_df_formatted[~wm_df_formatted['Program'].isin(not_list)]
# gsheet_uploader(wk, gsheet_df, append_df)

# sc_df_raw = all_longitudinal_card_import()
# sc_df_formatted = sc_formatter(sc_df_raw)
# not_list = gsheet_programs(gsheet_df)
# append_df = sc_df_formatted[~sc_df_formatted['Program'].isin(not_list)]
# gsheet_uploader(wk, gsheet_df, append_df)

# SC

# Requests

In [71]:
main_df[main_df['Dept Name'].str.contains('STATIONERY')]['Sub Dept Name'].unique()

array(['PERMANENT MARKERS', 'INK PENS', 'ACTIVITY KITS', 'STICKY NOTES'],
      dtype=object)

In [180]:


program_list

['ACWM180817S168_01_4.zip',
 'ACWM180817S168_02_4.zip',
 'ACWM190104S104_01_4.zip',
 'ACWM190104S104_02_4.zip',
 'ACWM190110C223_01_4.zip',
 'ACWM190110C223_02_4.zip',
 'ACWM190111C112_42_4.zip',
 'ACWM190111S106_01_4.zip',
 'ACWM190111S106_02_4.zip',
 'ACWM190111S112_01_4.zip',
 'ACWM190111S112_02_4.zip',
 'ACWM190111S115_01_4.zip',
 'ACWM190111S115_02_4.zip',
 'ACWM190118C113_44_4.zip',
 'ACWM190118S113_02_4.zip',
 'ACWM190125C110_43_4.zip',
 'ACWM190125S110_01_4.zip',
 'ACWM190201C103_42_4.zip',
 'ACWM190201C103_43_4.zip',
 'ACWM190201C103_45_4.zip',
 'ACWM190201S103_01_4.zip',
 'ACWM190208C114_42_4.zip',
 'ACWM190208C114_43_4.zip',
 'ACWM190208S114_02_4.zip',
 'ACWM190215C101_42_4.zip',
 'ACWM190215S101_02_4.zip',
 'ACWM190301C118_42_4.zip',
 'ACWM190301C124_42_4.zip',
 'ACWM190301C125_44_4.zip',
 'ACWM190301C131_44_4.zip',
 'ACWM190301C256_42_4.zip',
 'ACWM190301C256_44_4.zip',
 'ACWM190301S118_02_4.zip',
 'ACWM190301S124_02_4.zip',
 'ACWM190301S125_02_4.zip',
 'ACWM190301S131_02_

In [58]:
main_df = all_wm_metrics_mod.dropna(subset=['Placement Name'], how='all')
# main_df = main_df[main_df['Placement Name'].str.contains('Smartypants', case=False)]
# main_df = main_df[main_df['Program'].str.contains('_4.zip')]
# main_df = main_df[main_df['Weeks post'].str.contains('4W')]
main_df

12,Number,Program,Weeks post,start_date,end_date,Start date,End date,Item number,Placement Name,Dept,Sub Dept,Dept Name,Sub Dept Name,Freeosk stores,Rest of Chain stores,Traceable Audience,Non-Traceable Aud,Audience,Scans,Engagement,Immediate Conv,Wk 1 Conv,Wk 2 Conv,Wk 3 Conv,Wk 4 Conv,Wk 5 Conv,Wk 6 Conv,Wk 7 Conv,Wk 8 Conv,Wk 9 Conv,Wk 10 Conv,Wk 11 Conv,Wk 12 Conv,Delayed Conv,Total Conv,Repeat Conv,Repeat A Count,Repeat B Count,Repeat C Count,Conversion,Repeat,Repeat A%,Repeat B%,Repeat C%,Immediate,Delayed,PW A,W1 A,W2 A,W3 A,W4 A,W5 A,W6 A,W7 A,W8 A,W9 A,W10 A,W11 A,W12 A,PW B,W1 B,W2 B,W3 B,W4 B,W5 B,W6 B,W7 B,W8 B,W9 B,W10 B,W11 B,W12 B,PW C,W1 C,W2 C,W3 C,W4 C,W5 C,W6 C,W7 C,W8 C,W9 C,W10 C,W11 C,W12 C,A,B,C,Households,A%,B%,C%,A+B%,PW A,W1 A,W2 A,W3 A,W4 A,W5 A,W6 A,W7 A,W8 A,W9 A,W10 A,W11 A,W12 A,PW B,W1 B,W2 B,W3 B,W4 B,W5 B,W6 B,W7 B,W8 B,W9 B,W10 B,W11 B,W12 B,PW C,W1 C,W2 C,W3 C,W4 C,W5 C,W6 C,W7 C,W8 C,W9 C,W10 C,W11 C,W12 C,A $,B $,C $,Dollars,A $ %,B $ %,C $ %,A delay $,B delay $,C delay $,PW A,W1 A,W2 A,W3 A,W4 A,W5 A,W6 A,W7 A,W8 A,W9 A,W10 A,W11 A,W12 A,PW B,W1 B,W2 B,W3 B,W4 B,W5 B,W6 B,W7 B,W8 B,W9 B,W10 B,W11 B,W12 B,PW C,W1 C,W2 C,W3 C,W4 C,W5 C,W6 C,W7 C,W8 C,W9 C,W10 C,W11 C,W12 C,A units,B units,C units,Units,New Users Per Store,Total Users Per Store,New User $ Per Store,Total $ Per Store,Total Units Per Store
13,0,ACWM180817S168_01_4.zip,PW,'2018-08-17','2018-08-23',2018-08-17 00:00:00,2018-08-23 00:00:00,567992714,GoGo Squeeze BLASTZ,92,2628,GROCERY DRY GOODS,APPLESAUCE,198,1519,4164917,2.57629e+06,6.7412e+06,41132,0.00987583,1336,316,282,237,208,0,0,0,0,0,0,0,0,0,1336,16,0,0,0,0.0324808,0.011976,0,0,0,1,0,673,113,88,87,68,0,0,0,0,0,0,0,0,580,159,155,123,121,0,0,0,0,0,0,0,0,83,44,39,27,19,0,0,0,0,0,0,0,0,673,580,83,1336,0.503743,0.434132,0.0621257,0.937874,2193.24,423.36,353.04,329.78,328.59,0,0,0,0,0,0,0,0,1780.91,633.6,613.02,561.54,528.51,0,0,0,0,0,0,0,0,285.18,208.74,219.06,176.4,158.76,0,0,0,0,0,0,0,0,2193.24,1780.91,285.18,4259.33,0.514926,0.41812,0.0669542,0,0,0,746,144,124,113,112,0,0,0,0,0,0,0,0,606,216,209,191,180,0,0,0,0,0,0,0,0,97,71,75,60,54,0,0,0,0,0,0,0,0,746,606,97,1449,6.32828,6.74747,20.0715,21.5118,7.31818
14,0,ACWM180817S168_01_4.zip,4W,'2018-08-17','2018-08-23',2018-08-17 00:00:00,2018-08-23 00:00:00,567992714,GoGo Squeeze BLASTZ,92,2628,GROCERY DRY GOODS,APPLESAUCE,198,1519,4164917,2.57629e+06,6.7412e+06,41132,0.00987583,1336,316,282,237,208,0,0,0,0,0,0,0,0,1043,2379,281,0,0,0,0.0578382,0.118117,0,0,0,0.56158,0.43842,673,113,88,87,68,0,0,0,0,0,0,0,0,580,159,155,123,121,0,0,0,0,0,0,0,0,83,44,39,27,19,0,0,0,0,0,0,0,0,1029,1138,212,2379,0.432535,0.478352,0.0891131,0.910887,2193.24,423.36,353.04,329.78,328.59,0,0,0,0,0,0,0,0,1780.91,633.6,613.02,561.54,528.51,0,0,0,0,0,0,0,0,285.18,208.74,219.06,176.4,158.76,0,0,0,0,0,0,0,0,3628.01,4117.58,1048.14,8793.73,0.412568,0.46824,0.119192,1434.77,2336.67,762.96,746,144,124,113,112,0,0,0,0,0,0,0,0,606,216,209,191,180,0,0,0,0,0,0,0,0,97,71,75,60,54,0,0,0,0,0,0,0,0,1239,1402,357,2998,10.9444,12.0152,39.1191,44.4128,15.1414
15,0,ACWM180817S168_01_4.zip,12W,'2018-08-17','2018-08-23',2018-08-17 00:00:00,2018-08-23 00:00:00,567992714,GoGo Squeeze BLASTZ,92,2628,GROCERY DRY GOODS,APPLESAUCE,198,1519,4164917,2.57629e+06,6.7412e+06,41132,0.00987583,1336,316,282,237,208,0,0,0,0,0,0,0,0,1043,2379,281,0,0,0,0.0578382,0.118117,0,0,0,0.56158,0.43842,673,113,88,87,68,0,0,0,0,0,0,0,0,580,159,155,123,121,0,0,0,0,0,0,0,0,83,44,39,27,19,0,0,0,0,0,0,0,0,1029,1138,212,2379,0.432535,0.478352,0.0891131,0.910887,2193.24,423.36,353.04,329.78,328.59,0,0,0,0,0,0,0,0,1780.91,633.6,613.02,561.54,528.51,0,0,0,0,0,0,0,0,285.18,208.74,219.06,176.4,158.76,0,0,0,0,0,0,0,0,3628.01,4117.58,1048.14,8793.73,0.412568,0.46824,0.119192,1434.77,2336.67,762.96,746,144,124,113,112,0,0,0,0,0,0,0,0,606,216,209,191,180,0,0,0,0,0,0,0,0,97,71,75,60,54,0,0,0,0,0,0,0,0,1239,1402,357,2998,10.9444,12.0152,39.1191,44.4128,15.1414
16,1,ACWM180817S168_02_4.zip,PW,'2018-08-17','2018-08-23',20

In [46]:
main_df2 = main_df.merge(all_wm_lift_mod, how='left', left_on='Program', right_on='Program list')
main_df2.sort_values(by='Freeosk lift')

,Number,Program,Weeks post,start_date_x,end_date,Start date_x,End date,Item number,Placement Name,Dept,Sub Dept,Dept Name,Sub Dept Name,Freeosk stores_x,Rest of Chain stores,Traceable Audience,Non-Traceable Aud,Audience,Scans,Engagement,Immediate Conv,Wk 1 Conv,Wk 2 Conv,Wk 3 Conv,Wk 4 Conv,Wk 5 Conv,Wk 6 Conv,Wk 7 Conv,Wk 8 Conv,Wk 9 Conv,Wk 10 Conv,Wk 11 Conv,Wk 12 Conv,Delayed Conv,Total Conv,Repeat Conv,Repeat A Count,Repeat B Count,Repeat C Count,Conversion,Repeat,Repeat A%,Repeat B%,Repeat C%,Immediate,Delayed,PW A,W1 A,W2 A,W3 A,W4 A,W5 A,W6 A,W7 A,W8 A,W9 A,W10 A,W11 A,W12 A,PW B,W1 B,W2 B,W3 B,W4 B,W5 B,W6 B,W7 B,W8 B,W9 B,W10 B,W11 B,W12 B,PW C,W1 C,W2 C,W3 C,W4 C,W5 C,W6 C,W7 C,W8 C,W9 C,W10 C,W11 C,W12 C,A,B,C,Households,A%,B%,C%,A+B%,PW A,W1 A,W2 A,W3 A,W4 A,W5 A,W6 A,W7 A,W8 A,W9 A,W10 A,W11 A,W12 A,PW B,W1 B,W2 B,W3 B,W4 B,W5 B,W6 B,W7 B,W8 B,W9 B,W10 B,W11 B,W12 B,PW C,W1 C,W2 C,W3 C,W4 C,W5 C,W6 C,W7 C,W8 C,W9 C,W10 C,W11 C,W12 C,A $,B $,C $,Dollars,A $ %,B $ %,C $ %,A delay $,B delay $,C delay $,PW A,W1 A,W2 A,W3 A,W4 A,W5 A,W6 A,W7 A,W8 A,W9 A,W10 A,W11 A,W12 A,PW B,W1 B,W2 B,W3 B,W4 B,W5 B,W6 B,W7 B,W8 B,W9 B,W10 B,W11 B,W12 B,PW C,W1 C,W2 C,W3 C,W4 C,W5 C,W6 C,W7 C,W8 C,W9 C,W10 C,W11 C,W12 C,A units,B units,C units,Units,New Users Per Store,Total Users Per Store,New User $ Per Store,Total $ Per Store,Total Units Per Store,Program list,First row,Last row,Num rows,start_date_y,Start date_y,pre_period_end_st,Pre Period End,Week-1,Week-2,Week-3,Week-4,Freeosk Program Week,Week-1,Week-2,Week-3,Week-4,Non-Freeosk Program Week,Week-1,Week-2,Week-3,Week-4,Freeosk Prior 4,Control Prior 4,Freeosk Prior 4 per week,Control Prior 4 per week,Freeosk lift,Control lift,Freeosk stores_y,Control stores,F PW per club,F Prev4 per club,C PW per club,C Prev4 per club
5,268,ACWM191108C186_42_4.zip,4W,'2019-11-08','2019-11-14',2019-11-08 00:00:00,2019-11-14 00:00:00,"555135846, 555135843, 555135844, 554641667, 55...",WM SmartyPants Womens Nov 19,92* 40,"2767, 3658, 3660",GROCERY DRY GOODS* PHARMACY OTC,"PROBIOTICS VITAMINS, HOT CEREAL, MULTI VITAMINS",647,2858,10307313,4.68493e+06,1.49922e+07,72761,0.00705916,21760,11382,9272,9312,9349,0,0,0,0,0,0,0,0,39315,61075,6987,676,980,5331,0.839392,0.1144,0.0967511,0.14026,0.762988,0.356283,0.643717,7318,2611,2389,2317,2576,0,0,0,0,0,0,0,0,5874,3426,2701,3000,3176,0,0,0,0,0,0,0,0,8568,5345,4182,3995,3597,0,0,0,0,0,0,0,0,17211,18177,25687,61075,0.281801,0.297618,0.420581,0.579419,38148.9,11494.6,10971.2,10891.2,12788.8,0,0,0,0,0,0,0,0,28982.7,15291.8,12672.7,14192.6,15434.3,0,0,0,0,0,0,0,0,39508.2,26550.2,24069.7,26527.7,27732,0,0,0,0,0,0,0,0,84294.7,86574.1,144388,315257,0.267384,0.274615,0.458001,46145.8,57591.4,104880,8049,2899,2762,2807,3231,0,0,0,0,0,0,0,0,6298,3768,3119,3585,3972,0,0,0,0,0,0,0,0,10080,6930,6343,7111,7365,0,0,0,0,0,0,0,0,19748,20742,37829,78319,54.6955,94.3972,264.094,487.259,121.049,ACWM191108C186_42_4.zip,98722,99161,439,'2019-11-08',2019-11-08 00:00:00,'2019-11-07',2019-11-07 00:00:00,2019-11-01 00:00:00,2019-10-25 00:00:00,2019-10-18 00:00:00,2019-10-11 00:00:00,115420,107175,91528.7,92744.2,94324,287129,298182,246506,254815,262209,385772,1.06171e+06,96443,265428,0.196766,0.0817582,647,2858,178.392,149.062,100.465,92.8719
0,161,ACWM190719C454_42_4.zip,4W,'2019-07-19','2019-07-25',2019-07-19 00:00:00,2019-07-25 00:00:00,"555135846, 555135843, 555135844, 555135845",WM SmartyPants Womens Jul 19,40,"3658, 3660",PHARMACY OTC,"PROBIOTICS VITAMINS, MULTI VITAMINS",246,2455,4045291,1.86955e+06,5.91484e+06,40441,0.00999706,832,300,323,589,423,0,0,0,0,0,0,0,0,1635,2467,123,36,16,71,0.0610024,0.0498581,0.292683,0.130081,0.577236,0.337252,0.662748,509,134,132,303,224,0,0,0,0,0,0,0,0,151,65,65,143,102,0,0,0,0,0,0,0,0,172,101,126,143,97,0,0,0,0,0,0,0,0,1302,526,639,2467,0.527767,0.213214,0.259019,0.740981,7540.1,1973.74,2090.8,4823.72,3589.7,0,0,0,0,0,0,0,0,2236.8,961.38,1011.12,2189.36,1639.57,0,0,0,0,0,0,0,0,2481.66,1579.96,2212.56,2521.69,2006.39,0,0,0,0,0,0,0,0,20018.1,803

In [25]:
sc_df_metrics2 = sc_df_metrics.iloc[12:, :]
sc_df_metrics2.columns = sc_df_metrics2.iloc[0]
sc_df_metrics2 = sc_df_metrics2.drop(sc_df_metrics2.index[0])
sc_df_metrics2

# sc_df_metrics = sc_df_metrics.rename(columns={'Rest of Supercenters Stores': 'Rest of Chain stores'})

12,Number,Program,Weeks post,start_date,end_date,Start date,End date,Item number,Item description,Tracker description,Short description,Sample name,Short sample name,Category,Subcategory,Category Num,Subcat Num,Freeosk clubs,Control clubs,Audience,Transactions,Scans,Unique Scans,Engagement,Immediate Conv,Wks 1-4 Conv,Wks 5-8 Conv,Wks 9-12 Conv,Delayed Conv,Total Conv,Repeat Conv,Conversion,Repeat,Immediate,Delayed,PW A,1-4 A,5-8 A,9-12 A,PW B,1-4 B,5-8 B,9-12 B,PW C,1-4 C,5-8 C,9-12 C,A,B,C,Households,A%,B%,C%,A+B%,PW A,1-4 A,5-8 A,9-12 A,PW B,1-4 B,5-8 B,9-12 B,PW C,1-4 C,5-8 C,9-12 C,A $,B $,C $,Dollars,A $ %,B $ %,C $ %,A delay $,B delay $,C delay $,PW A,1-4 A,5-8 A,9-12 A,PW B,1-4 B,5-8 B,9-12 B,PW C,1-4 C,5-8 C,9-12 C,A units,B units,C units,Units
13,0,FSC18018_190736_20190524_0.zip,PW,'2019-05-24','2019-05-30',2019-05-24 00:00:00,2019-05-30 00:00:00,190736,AIRHEADS BITES,Airheads Fruit Bites,Airheads Fruit Bites,Airheads Variety Pack,Airheads Variety Pack,CANDY,CT SUGAR,1,3,569,15,10795394,6673144,797427,783380,0.119498,542,0,0,0,0,542,10,0.000691874,0.0184502,1,0,273,0,0,0,117,0,0,0,152,0,0,0,273,117,152,542,0.50369,0.215867,0.280443,0.719557,3987.99,0,0,0,1890.34,0,0,0,3147.36,0,0,0,3987.99,1890.34,3147.36,9025.69,0.441849,0.20944,0.348711,0,0,0,282,0,0,0,134,0,0,0,222,0,0,0,282,134,222,638
14,0,FSC18018_190736_20190524_0.zip,4W,'2019-05-24','2019-05-30',2019-05-24 00:00:00,2019-05-30 00:00:00,190736,AIRHEADS BITES,Airheads Fruit Bites,Airheads Fruit Bites,Airheads Variety Pack,Airheads Variety Pack,CANDY,CT SUGAR,1,3,569,15,10795394,6673144,797427,783380,0.119498,542,0,0,0,0,542,10,0.000691874,0.0184502,1,0,273,0,0,0,117,0,0,0,152,0,0,0,273,117,152,542,0.50369,0.215867,0.280443,0.719557,3987.99,0,0,0,1890.34,0,0,0,3147.36,0,0,0,3987.99,1890.34,3147.36,9025.69,0.441849,0.20944,0.348711,0,0,0,282,0,0,0,134,0,0,0,222,0,0,0,282,134,222,638
15,0,FSC18018_190736_20190524_0.zip,12W,'2019-05-24','2019-05-30',2019-05-24 00:00:00,2019-05-30 00:00:00,190736,AIRHEADS BITES,Airheads Fruit Bites,Airheads Fruit Bites,Airheads Variety Pack,Airheads Variety Pack,CANDY,CT SUGAR,1,3,569,15,10795394,6673144,797427,783380,0.119498,542,0,0,0,0,542,10,0.000691874,0.0184502,1,0,273,0,0,0,117,0,0,0,152,0,0,0,273,117,152,542,0.50369,0.215867,0.280443,0.719557,3987.99,0,0,0,1890.34,0,0,0,3147.36,0,0,0,3987.99,1890.34,3147.36,9025.69,0.441849,0.20944,0.348711,0,0,0,282,0,0,0,134,0,0,0,222,0,0,0,282,134,222,638
16,1,FSC18018_190736_20190524_12.zip,PW,'2019-05-24','2019-05-30',2019-05-24 00:00:00,2019-05-30 00:00:00,190736,AIRHEADS BITES,Airheads Fruit Bites,Airheads Fruit Bites,Airheads Variety Pack,Airheads Variety Pack,CANDY,CT SUGAR,1,3,569,15,10795399,6673144,797427,783379,0.119498,542,469,357,261,0,542,10,0.000691875,0.0184502,1,0,273,106,130,81,117,115,98,73,152,248,129,107,273,117,152,542,0.50369,0.215867,0.280443,0.719557,3987.99,2083.86,2612.99,1686.22,1890.34,2365.38,2222.56,1956.84,3147.36,8213.6,6731.3,6620.86,3987.99,1890.34,3147.36,9025.69,0.441849,0.20944,0.348711,0,0,0,282,147,185,119,134,167,158,138,222,580,475,467,282,134,222,638
17,1,FSC18018_190736_20190524_12.zip,4W,'2019-05-24','2019-05-30',2019-05-24 00:00:00,2019-05-30 00:00:00,190736,AIRHEADS BITES,Airheads Fruit Bites,Airheads Fruit Bites,Airheads Variety Pack,Airheads Variety Pack,CANDY,CT SUGAR,1,3,569,15,10795399,6673144,797427,783379,0.119498,542,469,357,261,469,1011,133,0.00129056,0.131553,0.536103,0.463897,273,106,130,81,117,115,98,73,152,248,129,107,379,232,400,1011,0.374876,0.229476,0.395648,0.604352,3987.99,2083.86,2612.99,1686.22,1890.34,2365.38,2222.56,1956.84,3147.36,8213.6,6731.3,6620.86,6071.85,4255.72,11361,21688.5,0.279957,0.19622,0.523823,2083.86,2365.38,8213.6,282,147,185,119,134,167,158,138,222,580,475,467,429,301,802,1532
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,.

In [26]:
sc_df_lift2 = sc_df_lift.iloc[4:, :]
sc_df_lift2.columns = sc_df_lift2.iloc[0]
sc_df_lift2 = sc_df_lift2.drop(sc_df_lift2.index[0])
sc_df_lift2
# sc_df_lift2 = sc_df_lift2.rename(columns={'Rest of Supercenters': 'Control stores'})

4,Program list,First row,Last row,Num rows,start_date,Start date,Week-1,Week-2,Week-3,Week-4,Freeosk Program Week,Week-1,Week-2,Week-3,Week-4,Non-Freeosk Program Week,Week-1,Week-2,Week-3,Week-4,Freeosk Prior 4,Control Prior 4,Freeosk Prior 4 per week,Control Prior 4 per week,Freeosk lift,Control lift,Freeosk clubs,Control clubs,F PW per club,F Prev4 per club,C PW per club,C Prev4 per club
5,FSC18018_190736_20190524_0.zip,1,741,740,'2019-05-24',2019-05-24 00:00:00,2019-05-17 00:00:00,2019-05-10 00:00:00,2019-05-03 00:00:00,2019-04-26 00:00:00,48845,52521.8,54215,56431.9,57935.4,1611.34,2070.28,1829.22,1999.38,1971.02,221104,7869.9,55276,1967.47,-0.116344,-0.181011,569,15,85.8435,97.1459,107.423,131.165
6,FSC18018_190736_20190524_12.zip,742,1650,908,'2019-05-24',2019-05-24 00:00:00,2019-05-17 00:00:00,2019-05-10 00:00:00,2019-05-03 00:00:00,2019-04-26 00:00:00,48845,52521.8,54215,56431.9,57935.4,1611.34,2070.28,1829.22,1999.38,1971.02,221104,7869.9,55276,1967.47,-0.116344,-0.181011,569,15,85.8435,97.1459,107.423,131.165
7,FSC18018_190736_20190524_4.zip,1651,2447,796,'2019-05-24',2019-05-24 00:00:00,2019-05-17 00:00:00,2019-05-10 00:00:00,2019-05-03 00:00:00,2019-04-26 00:00:00,48845,52521.8,54215,56431.9,57935.4,1611.34,2070.28,1829.22,1999.38,1971.02,221104,7869.9,55276,1967.48,-0.116344,-0.181011,569,15,85.8435,97.1459,107.423,131.165
8,FSC18018_190736_20190628_0.zip,2448,2820,372,'2019-06-28',2019-06-28 00:00:00,2019-06-21 00:00:00,2019-06-14 00:00:00,2019-06-07 00:00:00,2019-05-31 00:00:00,48115,51285.6,50625.6,55085,56677,0,0,0,0,0,213673,0,53418.3,0,-0.0992782,NaN,589,0,81.6893,90.6932,NaN,NaN
9,FSC18018_190736_20190628_12.zip,2821,3277,456,'2019-06-28',2019-06-28 00:00:00,2019-06-21 00:00:00,2019-06-14 00:00:00,2019-06-07 00:00:00,2019-05-31 00:00:00,48115,51285.6,50625.6,55085,56677,0,0,0,0,0,213673,0,53418.3,0,-0.0992782,NaN,589,0,81.6893,90.6932,NaN,NaN
10,FSC18018_190736_20190628_4.zip,3278,3678,400,'2019-06-28',2019-06-28 00:00:00,2019-06-21 00:00:00,2019-06-14 00:00:00,2019-06-07 00:00:00,2019-05-31 00:00:00,48115,51285.6,50625.6,55085,56677,0,0,0,0,0,213673,0,53418.3,0,-0.0992782,NaN,589,0,81.6893,90.6932,NaN,NaN
11,FSC18018_705387_20190524_0.zip,3679,4419,740,'2019-05-24',2019-05-24 00:00:00,2019-05-17 00:00:00,2019-05-10 00:00:00,2019-05-03 00:00:00,2019-04-26 00:00:00,200244,230964,229425,233996,241222,5881.74,6957.34,7927.56,6490.08,6873.48,935606,28248.5,233902,7062.11,-0.143898,-0.167142,569,20,351.922,411.075,294.087,353.106
12,FSC18018_705387_20190524_12.zip,4420,5328,908,'2019-05-24',2019-05-24 00:00:00,2019-05-17 00:00:00,2019-05-10 00:00:00,2019-05-03 00:00:00,2019-04-26 00:00:00,200244,230964,229425,233996,241222,5881.74,6957.34,7927.56,6490.08,6873.48,935606,28248.5,233902,7062.12,-0.143898,-0.167142,569,20,351.922,411.075,294.087,353.106
13,FSC18018_705387_20190524_4.zip,5329,6125,796,'2019-05-24',2019-05-24 00:00:00,2019-05-17 00:00:00,2019-05-10 00:00:00,2019-05-03 00:00:00,2019-04-26 00:00:00,200244,230964,229425,233996,241222,5881.74,6957.34,7927.56,6490.08,6873.48,935606,28248.5,233902,7062.12,-0.143898,-0.167142,569,20,351.922,411.075,294.087,353.106
14,FSC18018_705387_20190628_0.zip,6126,6498,372,'2019-06-28',2019-06-28 00:00:00,2019-06-21 00:00:00,2019-06-14 00:00:00,2019-06-07 00:00:00,2019-05-31 00:00:00,184234,198489,201495,217036,218071,0,0,0,0,0,835091,0,208773,0,-0.117537,NaN,589,0,312.792,354.453,NaN,NaN


In [50]:
sc_cat = sc_df2[sc_df2['Category'].str.contains('COFFEE AND BREAKFAST')]
sc_cat.Subcategory.unique()

array(['Adult Nutrition Bar', 'Kids Indulgent Bar', 'Tea', 'Truckload',
       'Creamers', 'K Cups'], dtype=object)

In [51]:
sc_cat = sc_df2[sc_df2['Category'].str.contains('OTC')]
sc_cat.Subcategory.unique()

array(['Analgesics', 'Laxatives', 'Joint Care', 'Adult MultiVitamins',
       'Child Multivitamins', 'Nutrition Bars', 'Omega Oils',
       'Letter Vitamins', 'Minerals', 'Protein Drinks & Powders',
       'Herbals/Supplements', 'Antacids', 'Diet Pills', 'Eye Care',
       'Cough Cold', 'Allergy'], dtype=object)

In [52]:
sc_cat = sc_df2[sc_df2['Category'].str.contains('Snacks')]
sc_cat.Subcategory.unique()

array(['Fruit Snacks Core', 'Cookies', 'Popcorn', 'Crackers - Mainstream',
       'On the Go - Crackers', 'Dried Fruit', 'Meat Snacks'], dtype=object)

In [54]:
sc_cat = sc_df2[sc_df2['Category'].str.contains('LAUNDRY AND HOME CARE')]
sc_cat.Subcategory.unique()

array(['Air Care', 'All Purpose Cleaners', 'Fabric Liquid Enhancers',
       'Liquid Laundry Detergents', 'Scent Boosters',
       'Disinfecting Wipes', 'Furniture Polish', 'Unit Dose Laundry',
       'Bathroom Cleaners', 'Sponges', 'e-business', 'Glass Cleaners',
       'Light Duty Dish Det'], dtype=object)

In [55]:
sc_cat = sc_df2[sc_df2['Category'].str.contains('HEALTH AND BEAUTY AIDS')]
sc_cat.Subcategory.unique()

array(['Bodywash', 'Skin Care', 'Mouthwash', 'Face Care', 'Hair Care',
       'Shaving', 'Toothpaste', 'Toothbrush', 'Liquid Soap'], dtype=object)

In [28]:
sc_df2.Category.unique()

array(['CANDY', 'HEALTH AND BEAUTY AIDS', 'LAUNDRY AND HOME CARE',
       'Snacks', 'OTC', 'PAPER GOODS', 'BABY CARE',
       'COFFEE AND BREAKFAST', 'TABLETOP AND BAGS', 'COOLER',
       'OIL - RICE - FRUITS - VEGETABLES', 'DELI PI', 'PET SUPPLIES',
       'FROZEN MEAT PI', 'JUICE - WATER - SPORT DRINKS',
       'Soda - Energy - Tea', 'PREPARED FOODS',
       'AUTOMOTIVE OIL AND CHEMICALS'], dtype=object)

In [73]:
sc_df2 = sc_df_metrics2.merge(sc_df_lift2, how='left', left_on='Program', right_on='Program list')

sc_df2 = sc_df2[sc_df2['Category'].str.contains('COFFEE AND BREAKFAST')]
sc_df2 = sc_df2[sc_df2['Program'].str.contains('_12.zip')]
sc_df2 = sc_df2[sc_df2['Weeks post'].str.contains('12W')]
sc_df2.sort_values(by='Freeosk lift', ascending = False)


,Number,Program,Weeks post,start_date_x,end_date,Start date_x,End date,Item number,Item description,Tracker description,Short description,Sample name,Short sample name,Category,Subcategory,Category Num,Subcat Num,Freeosk clubs_x,Control clubs_x,Audience,Transactions,Scans,Unique Scans,Engagement,Immediate Conv,Wks 1-4 Conv,Wks 5-8 Conv,Wks 9-12 Conv,Delayed Conv,Total Conv,Repeat Conv,Conversion,Repeat,Immediate,Delayed,PW A,1-4 A,5-8 A,9-12 A,PW B,1-4 B,5-8 B,9-12 B,PW C,1-4 C,5-8 C,9-12 C,A,B,C,Households,A%,B%,C%,A+B%,PW A,1-4 A,5-8 A,9-12 A,PW B,1-4 B,5-8 B,9-12 B,PW C,1-4 C,5-8 C,9-12 C,A $,B $,C $,Dollars,A $ %,B $ %,C $ %,A delay $,B delay $,C delay $,PW A,1-4 A,5-8 A,9-12 A,PW B,1-4 B,5-8 B,9-12 B,PW C,1-4 C,5-8 C,9-12 C,A units,B units,C units,Units,Program list,First row,Last row,Num rows,start_date_y,Start date_y,Week-1,Week-2,Week-3,Week-4,Freeosk Program Week,Week-1,Week-2,Week-3,Week-4,Non-Freeosk Program Week,Week-1,Week-2,Week-3,Week-4,Freeosk Prior 4,Control Prior 4,Freeosk Prior 4 per week,Control Prior 4 per week,Freeosk lift,Control lift,Freeosk clubs_y,Control clubs_y,F PW per club,F Prev4 per club,C PW per club,C Prev4 per club
1979,659,FSC19079_980222330_20200103_12.zip,12W,'2020-01-03','2020-01-09',2020-01-03 00:00:00,2020-01-09 00:00:00,980222330,SLEEPYTIME TEA 80 CT,2 40ct. Sleepytime Extra,2 40ct. Sleepytime Extra,2 40ct. Sleepytime Extra,2 40ct. Sleepytime Extra,COFFEE AND BREAKFAST,Tea,41,75,16,102,359904,220029,19795,19367,0.0899654,310,76,32,29,137,447,25,0.0230805,0.0559284,0.693512,0.306488,280,67,22,26,17,8,8,2,13,1,2,1,395,35,17,447,0.883669,0.0782998,0.0380313,0.961969,2207.91,574.92,281.07,253.29,131.52,60.9,71.82,23.94,103.77,102.27,109.65,229.35,3317.19,288.18,545.04,4150.41,0.799244,0.0694341,0.131322,1109.28,156.66,441.27,283,75,36,32,17,8,9,3,14,13,14,29,426,37,70,533,FSC19079_980222330_20200103_12.zip,451289,451939,650,'2020-01-03',2020-01-03 00:00:00,2019-12-27 00:00:00,2019-12-20 00:00:00,2019-12-13 00:00:00,2019-12-06 00:00:00,4483.86,1107.15,885.78,1013.46,941.64,7710.76,7652.82,5474.28,7341.6,7102.2,3948.03,27570.9,987.007,6892.72,3.54288,0.118681,16,102,280.241,61.688,75.5957,67.5757
350,116,FSC18095_990694_20181123_12.zip,12W,'2018-11-23','2018-11-29',2018-11-23 00:00:00,2018-11-29 00:00:00,990694,STEEP ORG LMNGNGR 60,Bigelow Organic Steep Lemon Ginger Tea,Bigelow Organic Steep Lemon Ginger,Bigelow Organic Steep Lemon Ginger Tea,Bigelow Organic Steep Lemon Ginger,COFFEE AND BREAKFAST,Tea,41,75,276,24,5268843,3259600,303140,298099,0.0929991,6192,2816,1229,911,4956,11148,1255,0.037397,0.112576,0.555436,0.444564,5560,2462,996,751,368,213,80,63,264,141,153,97,9769,724,655,11148,0.876301,0.0649444,0.0587549,0.941245,48099.4,25187.1,12957.6,10565.3,3478.6,2418.92,1386.53,1246.7,2581.34,2051.02,2493.61,2386.88,96809.4,8530.75,9512.85,114853,0.842898,0.0742754,0.0828263,48710,5052.15,6931.51,5809,3053,1572,1333,420,293,168,156,312,249,304,302,11767,1037,1167,13971,FSC18095_990694_20181123_12.zip,85756,86627,871,'2018-11-23',2018-11-23 00:00:00,2018-11-16 00:00:00,2018-11-09 00:00:00,2018-11-02 00:00:00,2018-10-26 00:00:00,107516,37324.4,42300.1,40529.7,39308.7,1887.84,2608.2,2239.6,2094.84,2144.52,159463,9087.16,39865.7,2271.79,1.69696,-0.169008,276,24,389.551,144.441,78.66,94.6579
1145,381,FSC19021_980088840_20190719_12.zip,12W,'2019-07-19','2019-07-25',2019-07-19 00:00:00,2019-07-25 00:00:00,980088840,KIND MINIS 32 CT,KIND Minis Variety Pack,KIND Minis Variety Pack,KIND Minis Variety Pack,KIND Minis Variety Pack,COFFEE AND BREAKFAST,Adult Nutrition Bar,41,2,300,287,5154356,3217835,445059,437233,0.13831,10549,3079,1701,1224,6004,16553,2644,0.0378585,0.159729,0.637286,0.362714,9305,2527,1319,935,366,121,100,72,878,431,282,217,14086,659,1808,16553,0.850964,0.0398115,0.109225,0.890775,156010,57378.7,39485.5,31429.7,6101.56,2640.24,2690.07,2350.64,16658.6,20505.3,20214.1,10567.1,284304,13782.5,67945.1,366032,0.77672,0.0376539,0.185626,128294,7680.95,51286.4,9514,3499,2408,1917,372,161,164